In [1]:
import pandas as pd
import datetime as dt

In [2]:
drop_cols = ['replies_count', 'retweets_count', 'likes_count', 'cashtags', 'conversation_id',
            'language', 'mentions', 'urls', 'photos', 'quote_url' ,'video' ,'thumbnail' ,'near',
            'geo', 'source', 'user_rt_id', 'retweet_id', 'reply_to', 'retweet_date', 'created_at',
            'translate', 'trans_src', 'trans_dest', 'place', 'name', 'timezone', 'user_rt', 'link', 'retweet']


In [3]:
df_biden = pd.read_csv("../data/tweets/all_biden_2020-11-02.csv").drop(columns=drop_cols)
df_trump = pd.read_csv("../data/tweets/all_trump_2020-11-02.csv").drop(columns=drop_cols)


/opt/conda/envs/twint2/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_trump['date'] = pd.to_datetime(df_trump['date'], format="%Y-%m-%d")
df_trump['time'] = pd.to_datetime(df_trump['time'],format='%H:%M:%S').dt.time
df_biden['date'] = pd.to_datetime(df_biden['date'], format="%Y-%m-%d")
df_biden['time'] = pd.to_datetime(df_biden['time'],format='%H:%M:%S').dt.time

df_trump.head(3)

,id,date,time,user_id,username,tweet,hashtags
0,1323414586586968064,2020-11-02,23:59:59,1242220312676241409,kamalatoe,Trump is campaigning with steel and autoworker...,"['macon', 'gasen', 'savannah', 'riseup']"
1,1323414586511429634,2020-11-02,23:59:59,2324659466,didih2214,Trump is having unscalable fencing erected aro...,['votehimout2020']
2,1323414586503143424,2020-11-02,23:59:59,215506541,cuffy__,"They support trump like he cares about them, h...",[]


In [5]:

dtypes = {
    'id': str,
    'user_id': str,
    'username': str,
    'tweet': str
}

df_trump = df_trump.astype(dtypes)
df_biden = df_biden.astype(dtypes)

In [6]:
df_biden.dtypes

id                  object
date        datetime64[ns]
time                object
user_id             object
username            object
tweet               object
hashtags            object
dtype: object

In [7]:
df_nov2 = pd.concat((pd.DataFrame(), df_trump, df_biden))

In [8]:
df_nov2.sample(10)

,id,date,time,user_id,username,tweet,hashtags
77599,1323401785952948225,2020-11-02,23:09:08,752490916577878016,door2theknown,@idigit1st @washingtonpost This was present un...,[]
146441,1323386662228021248,2020-11-02,22:09:02,1182170188550148096,dannydorito97,@benshapiro What's so wrong with having a pres...,[]
275742,1323358995810168832,2020-11-02,20:19:06,238856888,rlbraithwaite,@KirstenPowers The difference between Trump su...,[]
346275,1323414220453662721,2020-11-02,23:58:32,1322676282895835137,ohiopatriot12,⁦@realDonaldTrump⁩ ⁦@DonaldJTrumpJr⁩ Love Trum...,[]
271493,1323359943206952960,2020-11-02,20:22:51,1320533623917957122,mmhurley86,@AriMelber I have a feeling this nightmare wil...,[]
99732,1323396877492998144,2020-11-02,22:49:37,1266057892249001984,abrookmc1,@realDonaldTrump How ironic is it that Trump h...,[]
10663,1323412237751443461,2020-11-02,23:50:39,3028548812,movementcon,@SteveDeaceShow We assume they mean to predict...,[]
111546,1323394272981405705,2020-11-02,22:39:16,34137296,onyxgod32,Former Trump admin Homeland Security Ass Sec. ...,[]
6320,1323412785213198336,2020-11-02,23:52:50,1075078984138919936,michekatis,@StanysBujakera En lisant cet article vraiment...,[]
83179,1323389936544948225,2020-11-02,22:22:02,2909406949,arodd5151,Should we do a Biden train?,[]


In [9]:
df_nov2.drop_duplicates(subset='id', inplace=True)
df_nov2.shape

(359698, 7)

In [10]:
df_nov2['trump'] = df_nov2.tweet.str.contains("[Tt][Rr][Uu][Mm][Pp]")
df_nov2['biden'] = df_nov2.tweet.str.contains("[Bb][Ii][Dd][Ee][Nn]")
df_nov2.head(3)

,id,date,time,user_id,username,tweet,hashtags,trump,biden
0,1323414586586968064,2020-11-02,23:59:59,1242220312676241409,kamalatoe,Trump is campaigning with steel and autoworker...,"['macon', 'gasen', 'savannah', 'riseup']",True,False
1,1323414586511429634,2020-11-02,23:59:59,2324659466,didih2214,Trump is having unscalable fencing erected aro...,['votehimout2020'],True,False
2,1323414586503143424,2020-11-02,23:59:59,215506541,cuffy__,"They support trump like he cares about them, h...",[],True,False


In [11]:
df_nov2.trump.value_counts()

True     285919
False     73779
Name: trump, dtype: int64

In [12]:
df_nov2.biden.value_counts()

False    221981
True     137717
Name: biden, dtype: int64

## Now let's only include the same amount of tweets for each candidate.

We'll only use tweets that mention one candidate; and not the other. Let's balance these.

In [13]:
mask = (df_nov2.trump == 0) & (df_nov2.biden == 1)
biden_72k = df_nov2[mask].head(72000)

mask = (df_nov2.trump == 1) & (df_nov2.biden == 0)
trump_72k = df_nov2[mask].head(72000)

df_balanced = pd.concat((pd.DataFrame(), biden_72k, trump_72k))
df_balanced

,id,date,time,user_id,username,tweet,hashtags,trump,biden
181142,1323379284434669568,2020-11-02,21:39:43,2820503362,artistacriseida,All these articles showing that Biden is in th...,[],False,True
0,1323414585995526144,2020-11-02,23:59:59,1312487180258820096,annapieters17,@FoxNews Lady Gaga’s a nobody. Can’t figure ou...,[],False,True
4,1323414585232293888,2020-11-02,23:59:59,2335763630,kylechwatt,@The_Grupp “It is purely a fortuity that this ...,[],False,True
5,1323414585097908224,2020-11-02,23:59:59,3282973915,medianpolitics,@MSNBC What? When words just don't match rea...,[],False,True
6,1323414584930103298,2020-11-02,23:59:59,105937456,mannie9,Biden lives in confusion!,[],False,True
...,...,...,...,...,...,...,...,...,...
109201,1323394798229856256,2020-11-02,22:41:22,228320007,johnnylerose,Obama in Georgia has the last laugh at Trump -...,"['senators', 'voted', 'vote', 'senaterepubs']",True,False
109202,1323394797772689415,2020-11-02,22:41:21,886602582478729216,g0ldenstandard,@BritainElects Trump wins easily with at least...,[],True,False
109203,1323394797462315009,2020-11-02,22:41:21,1278448014915514368,panos_arlene,@TimMurtaugh @catturd2 @realDonaldTrump All I ...,[],True,False
109204,1323394797307105282,2020-11-02,22:41:21,3436140857,frankdiblasi860,Live: Trump holds 'MAGA' rally in Traverse Cit...,[],True,False


In [14]:
df_balanced.to_pickle("pickle/balanced_nov2_tweets.pick")